In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import torch
import numpy as np
from collections import defaultdict
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Two Stage Training lala
**Stage 1:** Feature Preservation

**Stage 2:** Domain Specialization

In [ ]:
model = YOLO("yolo11m.pt")

model.train(
    data="/content/drive/MyDrive/NRC-net/dataset/data.yaml",
    epochs=40,
    imgsz=640,
    batch=16,
    freeze=10,     # freeze backbone
    device=0,
    name="nrc_stage1",
    plots=True,
    verbose=False)


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/NRC-net/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=nrc_stage1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

KeyboardInterrupt: 

In [ ]:
model = YOLO("runs/detect/nrc_stage14/weights/best.pt")

model.train(
    data="/content/drive/MyDrive/NRC-net/dataset/data.yaml",
    epochs=80,
    imgsz=640,
    batch=16,
    freeze=0,
    device=0,
    resume=False,
    lr0=0.0001,
    name="nrc_stage2",
    plots=True,
    verbose=False

)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/NRC-net/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/nrc_stage14/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=nrc_stage2, nbs=64, nms=False, opset=None, optimize=False, optimizer=a

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a40dffe3920>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
class NRCRealTimeDetector:
    def __init__(self, model_path, conf_threshold=0.5):
        """
        Initialize the real-time NRC detector
        """
        # Load your trained model
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold

        # NRC region labels (customize based on your dataset)
        self.region_names = {
            0: "NRC Number",
            1: "Name",
            2: "Father's Name",
            3: "Birth Date",
            4: "Address",
            5: "Photo",
            6: "QR Code"
            # Add/modify based on your actual classes
        }

        # Colors for different regions (BGR format)
        self.colors = {
            "NRC Number": (0, 255, 0),      # Green
            "Name": (255, 0, 0),            # Blue
            "Father's Name": (0, 0, 255),    # Red
            "Birth Date": (255, 255, 0),     # Cyan
            "Address": (255, 0, 255),        # Magenta
            "Photo": (0, 255, 255),          # Yellow
            "QR Code": (128, 0, 128)         # Purple
        }

        # For FPS calculation
        self.fps_history = []
        self.last_time = time.time()

    def draw_detections(self, frame, results):
        """
        Draw bounding boxes and labels on the frame
        """
        if results[0].boxes is not None:
            for box in results[0].boxes:
                # Get coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Get confidence and class
                conf = float(box.conf[0].cpu().numpy())
                cls = int(box.cls[0].cpu().numpy())

                # Only show detections above confidence threshold
                if conf > self.conf_threshold:
                    # Get region name
                    region_name = self.region_names.get(cls, f"Class {cls}")
                    color = self.colors.get(region_name, (255, 255, 255))

                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Create label with confidence
                    label = f"{region_name}: {conf:.2f}"

                    # Calculate text size
                    (text_width, text_height), _ = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
                    )

                    # Draw background for text
                    cv2.rectangle(
                        frame,
                        (x1, y1 - text_height - 10),
                        (x1 + text_width + 10, y1),
                        color,
                        -1
                    )

                    # Draw text
                    cv2.putText(
                        frame,
                        label,
                        (x1 + 5, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 0, 0),
                        2
                    )

        return frame

    def draw_info_panel(self, frame, fps, detection_count):
        """
        Draw information panel on the frame
        """
        h, w = frame.shape[:2]

        # Semi-transparent overlay for info panel
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (300, 120), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)

        # Draw FPS
        cv2.putText(
            frame,
            f"FPS: {fps:.1f}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )

        # Draw detection count
        cv2.putText(
            frame,
            f"Regions Detected: {detection_count}",
            (20, 70),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (255, 255, 0),
            2
        )

        # Draw status
        status = "NRC Detected" if detection_count > 0 else "No NRC"
        color = (0, 255, 0) if detection_count > 0 else (0, 0, 255)
        cv2.putText(
            frame,
            status,
            (20, 100),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2
        )

        return frame

    def calculate_fps(self):
        """
        Calculate current FPS
        """
        current_time = time.time()
        fps = 1 / (current_time - self.last_time)
        self.last_time = current_time

        # Smooth FPS with moving average
        self.fps_history.append(fps)
        if len(self.fps_history) > 10:
            self.fps_history.pop(0)

        return sum(self.fps_history) / len(self.fps_history)

    def run(self, camera_id=0, display_width=1280, display_height=720):
        """
        Main loop for real-time detection
        """
        # Initialize camera
        cap = cv2.VideoCapture(camera_id)

        # Set camera properties for better performance
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, display_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, display_height)
        cap.set(cv2.CAP_PROP_FPS, 30)

        print("NRC Real-time Detector Started!")
        print("Controls:")
        print("  'q' - Quit")
        print("  '+' - Increase confidence threshold")
        print("  '-' - Decrease confidence threshold")
        print(f"Current confidence threshold: {self.conf_threshold}")

        while True:
            # Read frame from camera
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # Run inference
            results = self.model(frame, stream=True)

            # Get detection count
            detection_count = len(results[0].boxes) if results[0].boxes is not None else 0

            # Draw detections on frame
            frame = self.draw_detections(frame, results)

            # Calculate and draw FPS
            fps = self.calculate_fps()
            frame = self.draw_info_panel(frame, fps, detection_count)

            # Display the frame
            cv2.imshow('NRC Region Detection (Camo Camera)', frame)

            # Handle keyboard input
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('+') or key == ord('='):
                self.conf_threshold = min(1.0, self.conf_threshold + 0.05)
                print(f"Confidence threshold: {self.conf_threshold:.2f}")
            elif key == ord('-') or key == ord('_'):
                self.conf_threshold = max(0.1, self.conf_threshold - 0.05)
                print(f"Confidence threshold: {self.conf_threshold:.2f}")

        # Clean up
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
# Enhanced version with additional features
class AdvancedNRCRealTimeDetector(NRCRealTimeDetector):
    def __init__(self, model_path, conf_threshold=0.5):
        super().__init__(model_path, conf_threshold)
        self.detection_history = defaultdict(list)
        self.screenshot_counter = 0

    def save_detection(self, frame, results):
        """
        Save frame with detections as screenshot
        """
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        filename = f"nrc_detection_{timestamp}_{self.screenshot_counter}.jpg"
        cv2.imwrite(filename, frame)
        print(f"Screenshot saved: {filename}")
        self.screenshot_counter += 1

    def extract_region_data(self, frame, results):
        """
        Extract and display detected region data
        """
        if results[0].boxes is not None:
            print("\n=== Detected NRC Regions ===")
            for box in results[0].boxes:
                conf = float(box.conf[0].cpu().numpy())
                cls = int(box.cls[0].cpu().numpy())
                if conf > self.conf_threshold:
                    region_name = self.region_names.get(cls, f"Class {cls}")
                    print(f"  {region_name}: {conf:.2%} confidence")

                    # Store in history
                    self.detection_history[region_name].append({
                        'time': time.time(),
                        'confidence': conf
                    })
            print("===========================\n")

    def run(self, camera_id=0):
        """
        Enhanced main loop with additional features
        """
        cap = cv2.VideoCapture(camera_id)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        print("Advanced NRC Real-time Detector Started!")
        print("Controls:")
        print("  'q' - Quit")
        print("  's' - Save screenshot with detections")
        print("  'd' - Print detected regions to console")
        print("  '+'/'-' - Adjust confidence threshold")

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Run inference
            results = self.model(frame, stream=True)

            # Draw detections
            frame = self.draw_detections(frame, results)

            # Calculate and draw FPS
            fps = self.calculate_fps()
            detection_count = len(results[0].boxes) if results[0].boxes is not None else 0
            frame = self.draw_info_panel(frame, fps, detection_count)

            # Display
            cv2.imshow('Advanced NRC Detection (Camo)', frame)

            # Handle keys
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                self.save_detection(frame, results)
            elif key == ord('d'):
                self.extract_region_data(frame, results)
            elif key == ord('+'):
                self.conf_threshold = min(1.0, self.conf_threshold + 0.05)
                print(f"Confidence: {self.conf_threshold:.2f}")
            elif key == ord('-'):
                self.conf_threshold = max(0.1, self.conf_threshold - 0.05)
                print(f"Confidence: {self.conf_threshold:.2f}")

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    # Path to your trained model
    MODEL_PATH = "runs/detect/nrc_stage14/weights/best.pt"  # Adjust this path

    # Initialize and run detector
    detector = NRCRealTimeDetector(
        model_path=MODEL_PATH,
        conf_threshold=0.5  # Adjust based on your model's performance
    )

    # For advanced version with more features:
    # detector = AdvancedNRCRealTimeDetector(MODEL_PATH, conf_threshold=0.5)

    # Run with iPhone camera (Camo)
    # Camo typically appears as camera index 1 or 2
    detector.run(camera_id=1)  # Try 0, 1, or 2 for Camo